In [2]:
import ee

In [4]:
ee.Authenticate()

Enter verification code:  4/1AdQt8qhITDGG1wtjv4NRBE1GQLrHyW-r1nqyelLGkgu9cVT20XTT1EqO424



Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
import importlib
from flood_detection import modis
importlib.reload(modis)
from flood_detection.utils import misc
import geemap
from pathlib import Path
import rasterio
from pathlib import Path
import multiprocessing
from functools import partial

In [5]:
import HelperMethods.ExportChipsHelper as ExportHelper
importlib.reload(ExportHelper)

<module 'HelperMethods.ExportChipsHelper' from '/home/jovyan/work/HelperMethods/ExportChipsHelper.py'>

In [6]:
exportPath = Path('/Data/Raster/ModisHistoricalGFD/')

In [7]:
def downloadChip(year, chip, chipID, chip_i, fileList):
    name = chipID + "_" + str(chip_i) + "_" + str(year)
    if name in fileList:
        return
    
    geom = chip

    filename = exportPath/(name + '.tif')

    flood_map = modis.dfo(geom, str(year)+"-05-01", str(year)+"-11-30", "standard", '3Day', True)

    flood_map_slope_mask = misc.apply_slope_mask(flood_map, thresh=5)

    perm_water = misc.get_jrc_perm(geom)

    dfo_final = ee.Image(flood_map_slope_mask).addBands(perm_water)

    geemap.ee_export_image(dfo_final,\
                       filename,\
                       scale=250,\
                       crs='EPSG:4326',\
                       region=geom,\
                       file_per_band=False)

In [ ]:
fileList = ExportHelper.getFileList(exportPath)

chipIDs = ['0a', '0f', '10'] #'09', 
chips = ee.FeatureCollection("projects/ee-bobgiezi/assets/ShapeFiles/OverlapGrid")
for chipID in chipIDs:
    print('---- ' + str(chipID) + ' ----')
    fullChip = chips.filter(ee.Filter.eq('system:index', '000000000000000000' + chipID)).first()
    finerGrid = ExportHelper.createFineGrid(fullChip)
    listSize = finerGrid.size().getInfo()
    finerGridList = finerGrid.toList(listSize)
    for chip_i in range(listSize):
        print(f"processing chip {chip_i+1}/{listSize}")
        chip = ee.Feature(finerGridList.get(chip_i)).geometry()
        with multiprocessing.Pool(5) as pool:
            pool.map(partial(downloadChip, chip=chip, chipID=chipID, chip_i=chip_i), range(2001, 2022))

---- 0a ----
processing chip 1/64
Collected and pre-processed MODIS ImagesCollected and pre-processed MODIS Images
Collected and pre-processed MODIS ImagesCollected and pre-processed MODIS ImagesCollected and pre-processed MODIS Images



DFO Flood Dectection Complete
DFO Flood Dectection CompleteDFO Flood Dectection CompleteGenerating URL ...


Generating URL ...Generating URL ...
DFO Flood Dectection Complete

Generating URL ...
DFO Flood Dectection Complete
Generating URL ...
An error occurred while downloading.An error occurred while downloading.An error occurred while downloading.An error occurred while downloading.



('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
print('tes')

In [11]:
for name in [names[2]]:
    for year in range(2001, 2022):
        with rasterio.open(Path('DataExport')/name/(str(year)+'.tif'),'r+') as dst:
            dst.descriptions = tuple(['flooded', 'duration', 'clearViews', 'clearPerc', 'maxExtent', 'permanentWater'])